In [17]:
from allennlp_models.pretrained import load_predictor

In [18]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [19]:
from checklist.pred_wrapper import PredictorWrapper

In [20]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [21]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John met a friend and smiled.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'met',
   'description': '[ARG0: John] [V: met] [ARG1: a friend] and smiled .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
  {'verb': 'smiled',
   'description': '[ARG0: John] met a friend and [V: smiled] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['John', 'met', 'a', 'friend', 'and', 'smiled', '.']}

In [22]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [23]:
d = ["John met a friend and smiled."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'met',
     'description': '[ARG0: John] [V: met] [ARG1: a friend] and smiled .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
    {'verb': 'smiled',
     'description': '[ARG0: John] met a friend and [V: smiled] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['John', 'met', 'a', 'friend', 'and', 'smiled', '.']}],
 array([1.]))

In [24]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [25]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [26]:
# e = get_arg(pred,arg_target='ARG1')
# print(e)

In [27]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [28]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg1_people)

In [32]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("John met {first_name} {last_name} and smiled.", meta=True, nsamples=1000, remove_duplicates= True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Sue', 'last_name': 'Phillips'}, {'first_name': 'Dick', 'last_name': 'Hill'}, {'first_name': 'Emma', 'last_name': 'Harris'}, {'first_name': 'Sophie', 'last_name': 'Martin'}, {'first_name': 'Mary', 'last_name': 'Miller'}, {'first_name': 'Matthew', 'last_name': 'Murray'}, {'first_name': 'Steven', 'last_name': 'Hart'}, {'first_name': 'Melissa', 'last_name': 'Walker'}, {'first_name': 'Matt', 'last_name': 'Ryan'}, {'first_name': 'Victoria', 'last_name': 'Bell'}, {'first_name': 'Eric', 'last_name': 'Howard'}, {'first_name': 'Mark', 'last_name': 'Stevens'}, {'first_name': 'Elaine', 'last_name': 'Henderson'}, {'first_name': 'Heather', 'last_name': 'Stevens'}, {'first_name': 'Ralph', 'last_name': 'Kennedy'}, {'first_name': 'Sally', 'last_name': 'Brooks'}, {'first_name': 'Alex', 'last_name': 'Reed'}, {'first_name': 'Alison', 'last_name': 'Turner'}, {'first_name': 'Heather', 'last_name': 'Sullivan'}, {'first_name': 'Christopher', 'last_name': 'Smith'}, {'first_name': 'Thomas'

In [33]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 999 examples
Test cases:      999
Fails (rate):    5 (0.5%)

Example fails:
[ARG0: John] [V: met] [ARG1: Alice] Rose and smiled .
----
[ARG0: John] [V: met] [ARG1: Roger Rose and smiled] .
----
[ARG0: John] [V: met] [ARG1: Daniel Rose and smiled] .
----


In [35]:
import sys
#[https://howtodoinjava.com/examples/python-print-to-file/]
# Saving the reference of the standard output
original_stdout = sys.stdout  
 
with open('bert_result_items_argument_ellipsis.txt', 'w') as f:
    sys.stdout = f 
    for k, v in test.results.items():
        print(k, v, file = f)
    # Reset the standard output
    sys.stdout = original_stdout 